In [2]:
import sqlite3

import pycurl
try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO as BytesIO
try:
    from urllib.parse import urlencode
except ImportError:
    from urllib import urlencode
import urllib.request
import json
import time

import pypylon.pylon as pylon
from datetime import datetime
import time

In [3]:
con = sqlite3.connect('example.db')
cur = con.cursor()
cur.execute('''CREATE TABLE lidar_timestamps
               (time real, angle real)''')
cur.execute('''CREATE TABLE camera_timestamps
               (time real)''')
cur.close()

# Insert a row of data
# cur.execute(f"INSERT INTO lidar_timestamps VALUES ('{time}','{angle}')")
# cur.execute(f"INSERT INTO camera_timestamps VALUES ('{time}')")

# Save (commit) the changes
# con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
# con.close()

In [15]:
Base_URL = 'http://196.254.40.70/cgi/'

def sensor_do(s, url, pf, buf):
    s.setopt(s.URL, url)
    s.setopt(s.POSTFIELDS, pf)
    s.setopt(s.WRITEDATA, buf)
    s.perform()
    rcode = s.getinfo(s.RESPONSE_CODE)
    success = rcode in range(200, 207)
    print('%s %s: %d (%s)' % (url, pf, rcode, 'OK' if success else 'ERROR'))
    return success

In [16]:
sensor = pycurl.Curl()
buffer = BytesIO()
rc = sensor_do(sensor, Base_URL+'reset', urlencode({'data':'reset_system'}), buffer)
if rc:
    rc = sensor_do(sensor, Base_URL+'setting', urlencode({'rpm':'600'}), buffer)
if rc:
    rc = sensor_do(sensor, Base_URL+'setting', urlencode({'laser':'on'}), buffer)
sensor.close()

http://192.168.10.100/cgi/reset data=reset_system: 204 (OK)
http://192.168.10.100/cgi/setting rpm=600: 204 (OK)
http://192.168.10.100/cgi/setting laser=on: 204 (OK)


In [ ]:
from twisted.internet.protocol import DatagramProtocol
from twisted.internet import reactor

class ImageEventPrinter(pylon.ImageEventHandler):
    def OnImagesSkipped(self, camera, countOfSkippedImages):
        print("OnImagesSkipped event for device ", camera.GetDeviceInfo().GetModelName())
        print(countOfSkippedImages, " images have been skipped.")
        print()

    def OnImageGrabbed(self, camera, grabResult):
        print("OnImageGrabbed event for device ", camera.GetDeviceInfo().GetModelName())
        # Image grabbed successfully?
        if grabResult.GrabSucceeded():
            cur = con.cursor()
            cur.execute(f"INSERT INTO camera_timestamps VALUES ('{grabResult.ChunkTimestamp.Value}')")
            con.commit()
            cur.close()
            print("Timestamp: ", format_my_nanos(grabResult.ChunkTimestamp.Value))
            print()
        else:
            print("Error: ", grabResult.GetErrorCode(), grabResult.GetErrorDescription())
            
class Echo(DatagramProtocol):
    def datagramReceived(self, data, addr):
        angle = ((data[0x91 - 42] << 8) | data[0x90  - 42])
        time = ((data[0x04DD - 42] << 24) | (data[0x04DC - 42] << 16) | (data[0x04DB - 42] << 8) | data[0x04DA - 42])
        
        try:
            if self.old_angle > angle:
                if 36000 - self.old_angle < angle:
                    cur = con.cursor()
                    cur.execute(f"INSERT INTO lidar_timestamps VALUES ('{self.old_time}','{self.old_angle}')")
                    con.commit()
                    cur.close()
                    print(f"{self.old_time} {self.old_angle}")
                else:
                    cur = con.cursor()
                    cur.execute(f"INSERT INTO lidar_timestamps VALUES ('{time}','{angle}')")
                    con.commit()
                    cur.close()
                    print(f"{time} {angle}")
        except AttributeError:
            pass
        
        self.old_angle = angle
        self.old_time = time
        
tl_factory = pylon.TlFactory.GetInstance()

camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())

camera.Open()

camera.RegisterImageEventHandler(ImageEventPrinter(), pylon.RegistrationMode_ReplaceAll, pylon.Cleanup_Delete)

camera.GevSCPD.SetValue(1000)
camera.GevIEEE1588.SetValue(True)
camera.ChunkModeActive.SetValue(True)
camera.ChunkSelector.SetValue("Timestamp")
camera.ChunkEnable.SetValue(True)
camera.TriggerSelector.SetValue("FrameStart")
camera.TriggerMode.SetValue("On")

camera.StartGrabbing(pylon.GrabStrategy_OneByOne, pylon.GrabLoop_ProvidedByInstantCamera)

reactor.listenUDP(2368, Echo())
reactor.run(installSignalHandlers=False)

768990478 35858
OnImageGrabbed event for device  acA2040-35gc
769091338 168
769190870 35985
769290403 35794
769391263 92
769490796 35929
769590329 35780
769691188 115
769790721 35964
769890254 35802
769991114 110
770090647 35934
770190179 35768
770291039 89
770390572 35932
770490105 35775
770590965 89
770690498 35915
770791357 219
770890890 52
770990423 35882
771091283 183
771190816 6
771290348 35830
771391208 137
771490741 35967
771590274 35792
771691134 91
771790667 35909
771891526 212
771991059 43
772090592 35880
772191452 206
772290985 54
772390517 35899
772491377 217
772590910 58
772690443 35899
772791303 212
772890836 42
772990368 35865
773091228 163
773190761 35984
773290294 35808
773391154 112
773490686 35942
773590219 35770
773691079 75
773790612 35899
773891472 202
773991005 29
774090537 35868


Unhandled Error
Traceback (most recent call last):
  File "/Users/bobswinkels/Dropbox/Studie/Elektrotechniek HHS/2021-2022/Blok 4/Project/demo/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3338, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):
  File "/Users/bobswinkels/Dropbox/Studie/Elektrotechniek HHS/2021-2022/Blok 4/Project/demo/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/jv/4w2tv2854dlbrjmnn1w_s9jr0000gn/T/ipykernel_58305/1284493196.py", line 67, in <cell line: 67>
    reactor.run(installSignalHandlers=False)
  File "/Users/bobswinkels/Dropbox/Studie/Elektrotechniek HHS/2021-2022/Blok 4/Project/demo/.venv/lib/python3.9/site-packages/twisted/internet/base.py", line 1315, in run
    self.mainLoop()
--- <exception caught here> ---
  File "/Users/bobswinkels/Dropbox/Studie/Elektrotechniek HHS/2021-2022/Blok 4/Proj

774191397 189
774290930 26
774390463 35855
774491323 166
774590855 11
774690388 35866
774791248 198
774890781 43
774990314 35874
775091174 171
775190706 35988
775290239 35808
775391099 111
775490632 35941
775590165 35765
775691024 62
775790557 35877
775891417 177
775990950 11
776090483 35846
776191343 152
776290875 35975
776390408 35805
776491268 118
776590801 35959
776690334 35809
776791193 131
776890726 35966
776990259 35795
777091119 112
777190652 35953
777290184 35791
777391044 102
777490577 35928
777591437 232
777690970 60
777790503 35889
777891362 191
777990895 24
778090428 35862
778191288 174
778290821 3
778390353 35827
778491213 128
778590746 35950
778690279 35780
778791139 95
778890672 35934
778990204 35780
779091064 103
779190597 35945
779290130 35786
779390990 104
779490522 35934
779591382 234
779690915 59
779790448 35893
779891308 208
779990841 42
780090373 35871
780191233 168
780290766 35982
780390299 35801
780491159 108
780590691 35938
780690224 35770
780791084 81


In [4]:
camera.Close()

In [8]:
sensor = pycurl.Curl()
buffer = BytesIO()
rc = sensor_do(sensor, Base_URL+'reset', urlencode({'data':'reset_system'}), buffer)
sensor.close()

http://192.168.10.100/cgi/reset data=reset_system: 204 (OK)


In [5]:
con.close()